In [1]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



import os

from sklearn.model_selection import train_test_split



['ratings_Electronics.csv']


In [2]:
ele=pd.read_csv('ratings_Electronics.csv',names=['reviewerID','Product_ID','rating','timestamp'])

In [3]:
ele.shape

(7824482, 4)

In [4]:
#remove timestamp
ele=ele.drop(columns='timestamp')
ele.head()

,reviewerID,Product_ID,rating
0,AKM1MP6P0OYPR,0132793040,5.0
1,A2CX7LUOHB2NDG,0321732944,5.0
2,A2NWSAGRHCP8N5,0439886341,1.0
3,A2WNBOD3WNDNKT,0439886341,3.0
4,A1GI0U4ZRJA8WN,0439886341,1.0


In [5]:
ele['reviewerID'].unique().shape

(4201696,)

In [6]:
ele.isnull().sum()

reviewerID    0
Product_ID    0
rating        0
dtype: int64

In [7]:
above_fifty=ele.groupby(['reviewerID']).agg('count').sort_values(by='rating',ascending=False)

In [8]:
above_fifty_id=above_fifty[above_fifty['rating']>50].index

In [9]:
above_fifty_id

Index(['A5JLAU2ARJ0BO', 'ADLVFFE4VBT8', 'A3OXHLG6DIBRW8', 'A6FIAB28IS79',
       'A680RUE1FDO8B', 'A1ODOGXEYECQQ8', 'A36K2N527TXXJN', 'A2AY4YUOX2N1BQ',
       'AWPODHOB4GFWL', 'A25C2M3QF9G7OQ',
       ...
       'AWPN47SSWK1JV', 'ABMNX856X89CS', 'A1AKW788238PWQ', 'AEIB3UR05OP7A',
       'A1AZQCVCDBQP6', 'AN2IZT80ZUNJN', 'A18NDN2CIG2TKR', 'A2ZRMCRLDSUX6R',
       'ASF3J4UYV34KA', 'ACP9S40H3SHY1'],
      dtype='object', name='reviewerID', length=1466)

In [10]:
Final_df=ele[ele['reviewerID'].isin(above_fifty_id.values)]

In [11]:
Final_df.shape

(122171, 3)

In [12]:
Final_df=(Final_df.reset_index())

In [13]:
Final_df=Final_df.drop(columns='index')

In [14]:
Final_df.head()

,reviewerID,Product_ID,rating
0,AT09WGFUM934H,0594481813,3.0
1,A32HSNCNPRUMTR,0970407998,1.0
2,A17HMM1M7T9PJ1,0970407998,4.0
3,A3CLWR1UUZT6TG,0972683275,5.0
4,A3TAS1AG6FMBQW,0972683275,5.0


In [15]:
Train,Test=train_test_split(Final_df,train_size=0.75,random_state=1)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [16]:
Train.shape

(91628, 3)

In [17]:
R_df=Train.pivot(index = 'reviewerID', columns ='Product_ID', values = 'rating').fillna(0)

In [18]:
R_df.head()

Product_ID,0594481813,0970407998,0972683275,1400501466,1400501520,1400532620,1400532655,140053271X,1400599997,1400698987,1400699169,3744295508,7562434166,787988002X,8862935307,8862936826,9043413585,9573212919,9575871979,9800359788,9981719005,9981739588,9983722615,998376234X,9983891212,9984922626,9985511298,9985696999,9989360391,B000000O48,B000001OKH,B000001OM4,B000001OM5,B000001OMI,B000001OMN,B000001OMZ,B000001ON0,B000001ON6,B000006OQ8,B00000DM9W,...,B00KOHQU58,B00KOLNVAQ,B00KONCDVM,B00KOUIC0Q,B00KPRWAX8,B00KPXQQK0,B00KRI1AP4,B00KSBB84S,B00KSTODWY,B00KT17TZ4,B00KU4RD4M,B00KVLY8E2,B00KVNY2KA,B00KW2W6B2,B00KW3OFG0,B00KWHMR6G,B00KWL62QI,B00KWMNDDM,B00KWWY1PQ,B00KXAFYZS,B00KXMN08E,B00KXUHX9S,B00KYMCJF8,B00L0YFJHY,B00L1I727Y,B00L1I80K2,B00L1NZTSS,B00L21HC7A,B00L2442H0,B00L26YDA4,B00L2P3TRS,B00L3YHF6O,B00L403O94,B00L8I6SFY,B00LA6T0LS,B00LED02VY,B00LGN7Y3G,B00LGQ6HL8,B00LI4ZZO8,B00LKG1MC8
reviewerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A100UD67AHFODS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A100WO06OQR8BQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A105S56ODHGJEK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A105TOJ6LTVMBG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10AFVU66A79Y1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_df, k = 50)

In [20]:
sigma = np.diag(sigma)

In [21]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

In [22]:
#lets find the recomondation for reviewerID=AT09WGFUM934H
reviewerID='A32HSNCNPRUMTR'
user_id = 2

In [23]:
 # Get and sort the user's predictions
user_row_number = user_id - 1 # UserID starts at 1, not 0
sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False)

In [24]:
##Top 5 Product recmondations is 
sorted_user_predictions.head(5).index.values

array(['B003ES5ZUU', 'B0088CJT4U', 'B008DWCRQW', 'B002R5AM7C',
       'B004CLYEE6'], dtype=object)

# Popularity based recomondation

In [25]:
class popularity_recommender_py():
    def __init__(self):
        self.train_data = None
        self.user_id = None
        self.item_id = None
        self.popularity_recommendations = None
        
    #Create the popularity based recommender system model
    def create(self, train_data, user_id, item_id):
        self.train_data = train_data
        self.user_id = user_id
        self.item_id = item_id

        #Get a count of user_ids for each unique song as recommendation score
        train_data_grouped = train_data.groupby([self.item_id]).agg({self.user_id: 'count'}).reset_index()
        train_data_grouped.rename(columns = {'reviewerID': 'score'},inplace=True)
        print(train_data_grouped.columns)
        #Sort the songs based upon recommendation score
        train_data_sort = train_data_grouped.sort_values(['score', self.item_id], ascending = [0,1])
    
        #Generate a recommendation rank based upon score
        train_data_sort['Rank'] = train_data_sort['score'].rank(ascending=0, method='first')
        
        #Get the top 10 recommendations
        self.popularity_recommendations = train_data_sort.head(10)

    #Use the popularity based recommender system model to
    #make recommendations
    def recommend(self, user_id):    
        user_recommendations = self.popularity_recommendations
        
        #Add user_id column for which the recommendations are being generated
        user_recommendations['user_id'] = user_id
    
        #Bring user_id column to the front
        cols = user_recommendations.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        user_recommendations = user_recommendations[cols]
        
        return user_recommendations

In [26]:
pm = popularity_recommender_py()
pm.create(Train, 'reviewerID', 'Product_ID')

Index(['Product_ID', 'score'], dtype='object')


In [27]:
reviewr_id='A32HSNCNPRUMTR'
pm.recommend(reviewr_id)

,user_id,Product_ID,score,Rank
31467,A32HSNCNPRUMTR,B0088CJT4U,150,1.0
19943,A32HSNCNPRUMTR,B003ES5ZUU,133,2.0
8784,A32HSNCNPRUMTR,B000N99BBC,130,3.0
30864,A32HSNCNPRUMTR,B007WTAJTO,123,4.0
31164,A32HSNCNPRUMTR,B00829TIEK,114,5.0
31741,A32HSNCNPRUMTR,B008DWCRQW,104,6.0
23177,A32HSNCNPRUMTR,B004CLYEDC,98,7.0
17637,A32HSNCNPRUMTR,B002R5AM7C,94,8.0
31160,A32HSNCNPRUMTR,B00829THK0,93,9.0
24966,A32HSNCNPRUMTR,B004T9RR6I,87,10.0
